In [3]:
import os
import oss2
import gzip
from itertools import islice
import json
import pandas as pd
import urllib3
import logging
import sys
from datetime import datetime

In [138]:
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO)

arguments=sys.argv
logging.info("Arguments length: " + str(len(arguments)) + ". Argurments: " + str(arguments))
if len(arguments) <= 1:
    print("usage: {} env. For example, {} test 20181220".format(arguments[0], arguments[0]))
    sys.exit(-1)

env=arguments[1]
if env == "prod":
    endpoint="oss-cn-shanghai-internal.aliyuncs.com"
    postProfileUrl="http://gateway.fnvalley.com/profile-service/2.0.0/extra_profile/saveIosDeviceTokenList"
    kafkaOssPathPrefix="kafka-bakcup/{}/event-prod"
elif env == "test":
    endpoint="oss-cn-shanghai-internal.aliyuncs.com"
    postProfileUrl="http://testgate.fnvalley.com/profile-service/2.0.0/extra_profile/saveIosDeviceTokenList"
    kafkaOssPathPrefix="kafka-bakcup-test/{}/event-test"
else:
    endpoint="oss-cn-shanghai.aliyuncs.com"
    postProfileUrl="http://testgate.fnvalley.com/profile-service/2.0.0/extra_profile/saveIosDeviceTokenList"
    kafkaOssPathPrefix="kafka-bakcup-test/{}/event-test"

if len(arguments) >= 3:
    date=arguments[2]
else:
    date=datetime.now().strftime("%Y%m%d")
    
accessKeyID="LTAItd3meVs30UzA"
accessKeySecret="mt9g18CLp18gqXcaWxLSZX4VqX1mH4"

internalBucket="internal-fnvalley"
ACCESS_TOKEN="4f55964d-d361-44fe-94d8-2f2eaedce16e"
loginIdDeviceTokenMap = {}

logging.info("Oss endpoint: " + endpoint)
logging.info("Collecting date: " + date)

2018-12-17 10:12:06,217 Arguments length: 3. Argurments: ['/ws/service/venv/py3/lib/python3.7/site-packages/ipykernel_launcher.py', '-f', '/Users/bell/Library/Jupyter/runtime/kernel-3f5b1704-dbb3-4fd2-b181-654fcae0b2c8.json']


In [139]:
auth = oss2.Auth(accessKeyID, accessKeySecret)
bucket = oss2.Bucket(auth, 'http://' + endpoint, internalBucket)
http = urllib3.PoolManager()

2018-12-17 10:12:45,907 oss2.api [INFO] 140735715738496 : Init oss bucket, endpoint: http://oss-cn-shanghai.aliyuncs.com, isCname: False, connect_timeout: None, app_name: , enabled_crc: True
2018-12-17 10:12:45,907 Init oss bucket, endpoint: http://oss-cn-shanghai.aliyuncs.com, isCname: False, connect_timeout: None, app_name: , enabled_crc: True


In [140]:
def checkAndAddDeviceToken(event):
    try:
        eventJson = json.loads(event)
        if eventJson.get("deviceToken") != None and eventJson.get("loginId") != None:
            loginIdDeviceTokenMap[eventJson.get("loginId")] = eventJson.get("deviceToken")
    except Exception as e:
        logging.info("Event: [" + event + "], Exeption: " + str(e))

In [117]:
o = bucket.list_objects(prefix=kafkaOssPathPrefix.format(date))
for d in o.object_list:
    object_stream = bucket.get_object(d.key)
    data = gzip.decompress(object_stream.read())
    for event in data.decode('utf8').split("\r\n"):
        checkAndAddDeviceToken(event)

2018-12-14 19:01:06,099 oss2.api [INFO] 140735715738496 : Start to List objects, bucket: internal-fnvalley, prefix: kafka-bakcup/20181113/event-prod, delimiter: , marker: , max-keys: 100
2018-12-14 19:01:06,099 Start to List objects, bucket: internal-fnvalley, prefix: kafka-bakcup/20181113/event-prod, delimiter: , marker: , max-keys: 100
2018-12-14 19:01:06,225 oss2.api [INFO] 140735715738496 : List objects done, req_id: 5C138D728DA7CA8C299E874C, status_code: 200
2018-12-14 19:01:06,225 List objects done, req_id: 5C138D728DA7CA8C299E874C, status_code: 200
2018-12-14 19:01:06,231 oss2.api [INFO] 140735715738496 : Start to get object, bucket: internal-fnvalley， key: kafka-bakcup/20181113/event-prod.0.gz, range: , headers: {}, params: {}
2018-12-14 19:01:06,231 Start to get object, bucket: internal-fnvalley， key: kafka-bakcup/20181113/event-prod.0.gz, range: , headers: {}, params: {}
2018-12-14 19:01:06,273 oss2.api [INFO] 140735715738496 : Get object done, req_id: 5C138D728DA7CA8C299E87B

In [120]:
logging.info("device token map: " + str(loginIdDeviceTokenMap))
if len(loginIdDeviceTokenMap) > 0:
    dataList = []
    for loginId in loginIdDeviceTokenMap: 
        data = {}
        data["loginId"] = loginId
        data["iosDeviceToken"] = loginIdDeviceTokenMap.get(loginId)
        if data["iosDeviceToken"] != "":
            dataList.append(data)
    response = http.request("POST", postProfileUrl, body = json.dumps(dataList), headers = {"Content-Type":"application/json", "Authorization": "Bearer " + ACCESS_TOKEN})
    logging.info("Response status: " + str(response.status))
    if response.status != 200:
        sys.exit(-1)

[{"loginId": "18321593357", "iosDeviceToken": "8d5bf8557bfe31aa4e09578e75bee5089c73a2782a2d0578b9d241cabeeb71ba"}]
